# ML Summer School - Assignment 4

In this assignment, we will be clustering movies on the basis of their synopsis. Unlike previous assignments, this time you will be implementing the KMeans clustering algorithm from scratch.

### Load Libraries

In [39]:
import numpy as np
import pandas as pd
import nltk
import re
import os
from sklearn import feature_extraction
import pickle
import scipy
import random 

In [17]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [70]:
from collections import Counter

In [18]:
from nltk.stem import WordNetLemmatizer


### Load data
We will be using the synopsis of Top 100 movies ranked by IMDb. `titles`, `genres` and `synopsis` are lists stored as pickled files. We first load them into memory. 

Take a look at the lists we have just loaded.

In [7]:
titles = pickle.load(open('titles.pkl','rb'))
genres = pickle.load(open('genres.pkl','rb'))
synopsis = pickle.load(open('synopses.pkl','rb'))
print(len(titles), len(genres), len(synopsis))

100 100 100


### Preprocessing the Data

Let us take a look at some of the synopsis.

In [8]:
print(synopsis[0][:1000])

 Plot  [edit]  [  [  edit  edit  ]  ]  
  On the day of his only daughter's wedding, Vito Corleone hears requests in his role as the Godfather, the Don of a New York crime family. Vito's youngest son, Michael, in a Marine Corps uniform, introduces his girlfriend, Kay Adams, to his family at the sprawling reception. Vito's godson Johnny Fontane, a popular singer, pleads for help in securing a coveted movie role, so Vito dispatches his consigliere, Tom Hagen, to Los Angeles to influence the abrasive studio head, Jack Woltz. Woltz is unmoved until the morning he wakes up in bed with the severed head of his prized stallion.  On the day of his only daughter's wedding,   Vito Corleone  Vito Corleone   hears requests in his role as the Godfather, the   Don  Don   of a New York crime family. Vito's youngest son,   Michael  Michael  , in a   Marine Corps  Marine Corps   uniform, introduces his girlfriend,   Kay Adams  Kay Adams  , to his family at the sprawling reception. Vito's godson   Johnny

In [9]:
print(synopsis[1][:1000])

 Plot  [edit]  [  [  edit  edit  ]  ]  
  In 1947, banker Andy Dufresne is convicted of murdering his wife and her lover and sentenced to two consecutive life sentences at the fictional Shawshank State Penitentiary in the state of Maine. Andy befriends contraband smuggler Ellis "Red" Redding, an inmate serving a life sentence. Red procures a rock hammer and later a large poster of Rita Hayworth for Andy. Working in the prison laundry, Andy is regularly assaulted by the "bull queer" gang "the Sisters" and their leader, Bogs.  In 1947, banker Andy Dufresne is convicted of murdering his wife and her lover and sentenced to two consecutive life sentences at the fictional Shawshank State Penitentiary in the state of Maine. Andy befriends   contraband  contraband   smuggler Ellis "Red" Redding, an inmate serving a life sentence. Red procures a   rock hammer  rock hammer   and later a large poster of   Rita Hayworth  Rita Hayworth   for Andy. Working in the prison laundry, Andy is regularly as

We can see that the synopsis contain a lot of names (Proper Nouns) and years. They are specific to the movies and do not seem to be helpful to cluster the movies. We should remove them

We will also be lemmatizing the words. [Lemmatizing](https://www.twinword.com/blog/what-is-lemmatization/ ) refers to replacing word by its base form (lemma). For example, lemmatizing `cars` gives `car`. Lemmatizing will help in clubbing all different inflections of words, eg. `loves`, `loving`, `loved` will all be lemmatized to `love`.

### Task 1

Complete the following function `preprocess_data`. It takes in text as input, and [removes proper nouns](https://stackoverflow.com/questions/39634222/is-there-a-way-to-remove-proper-nouns-from-a-sentence-using-python), non-alphabetic words, lemmatizes the data and lower cases the entire text.

In [83]:
def preprocess_data(text):
    
    """Removes proper noun, non-alphabetic words, lemmatizes the data and lower cases the entire text"""
    lexicon=[]
    lemmatizer=WordNetLemmatizer()
    lexicon=[]
    all_words=word_tokenize(text)
    #print(all_words)
    lexicon=list(all_words)
    lexicon=[lemmatizer.lemmatize(i) for i in lexicon]
   # w_count=Counter(lexicon)
    #l2=[]
    #for w in w_count.values():
     #   if w<1000 and w>5:
     #       l2.append(w)
    text=(" ".join(lexicon))
    text=text.lower()
    
    return text

**Sample input**: preprocess_data(synopsis[0][:1000])


**Sample output**: 'edit edit edit day daughter wedding hear request role crime family son uniform introduce girlfriend family reception godson singer pleads help secure movie role dispatch consigliere influence studio head be unmoved morning wake bed head stallion day daughter wedding hear request role crime family son uniform introduce girlfriend family reception godson'



In [84]:
preprocess_data(synopsis[0][:1000])

"plot [ edit ] [ [ edit edit ] ] on the day of his only daughter 's wedding , vito corleone hears request in his role a the godfather , the don of a new york crime family . vito 's youngest son , michael , in a marine corps uniform , introduces his girlfriend , kay adams , to his family at the sprawling reception . vito 's godson johnny fontane , a popular singer , pleads for help in securing a coveted movie role , so vito dispatch his consigliere , tom hagen , to los angeles to influence the abrasive studio head , jack woltz . woltz is unmoved until the morning he wake up in bed with the severed head of his prized stallion . on the day of his only daughter 's wedding , vito corleone vito corleone hears request in his role a the godfather , the don don of a new york crime family . vito 's youngest son , michael michael , in a marine corps marine corps uniform , introduces his girlfriend , kay adams kay adams , to his family at the sprawling reception . vito 's godson johnny"

### Creating the Features

We will extract features from the synopsis using TfIdf Vectoriser, which will be used to cluster the movies. 

Read more about TfIdf Vectoriser [here](http://blog.christianperone.com/2011/09/machine-learning-text-feature-extraction-tf-idf-part-i/) and [here](http://blog.christianperone.com/2011/10/machine-learning-text-feature-extraction-tf-idf-part-ii/).

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=20000, min_df=0.2,\
                                   stop_words='english', preprocessor=preprocess_data, \
                                   use_idf=True, ngram_range=(1, 1))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopsis)

print(tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()

Wall time: 5.74 s
(100, 473)


In [86]:
tfidf_matrix

<100x473 sparse matrix of type '<class 'numpy.float64'>'
	with 16133 stored elements in Compressed Sparse Row format>

### Task 2 - Implementing KMeans

This is the most important task of this assignment because in this, you will implement your own K-Means. Fill in the function `KMeans` which will return the labels and cluster centers in the tuple `(labels,centers)` for the given feature matrix `X` and `num_clusters`. We also pass the `max_iter` parameter to run KMeans for that many iterations as it sometimes gets stuck on a local minima. You can learn more about Kmeans clustering [here](https://en.wikipedia.org/wiki/K-means_clustering) and [here](https://www.datascience.com/blog/k-means-clustering).

**Sample Input**: `[[2,1], [2,3], [8,1], [8,3]]`, `num_clusters=2`

**Sample Output**: `([0,0,1,1], [[2,2], [8,2]])`

In [154]:
x=np.array([[2,1], [2,3], [8,1], [8,3]])
x=x.tolist()
centers=random.sample(x,k=2)
print(centers)
x=np.asarray(x)
centers=np.asarray(centers)
#print(centers,x)
for i in x:
    print(i[0])
print(x[0])

[[2, 1], [8, 1]]
2
2
8
8
[2 1]


In [172]:
def Kmeans(X, num_clusters=8, max_iter=300):
    
    for iter in range(max_iter):
        X=X.tolist()
        centers=random.sample(X,k=num_clusters)
        X=np.asarray(X)
        centers=np.asarray(centers)
        labels=[]
        dist=[]
        for i in X:
            for c in centers:
                d=((i[0]-c[0])**2)+(i[1]-c[1])**2
                dist.append(d)
            index,element=min(list(enumerate(dist)),key=lambda x:x[1])    
            labels.append(index)        
        for i in range(num_clusters):
            sum=[0,0]
            counter=0
            t=0
            for l in labels:
                if(l==i):
                   
                    sum=sum+X[t]
                    counter=counter+1
                t=t+1
            if counter==0 :
                continue
            sum[0]=sum[0]/counter
            sum[1]=sum[1]/counter
            centers[i]=sum
    return (labels, centers)

In [173]:
Kmeans(np.array([[2,1], [2,3], [8,1], [8,3]]), num_clusters=2)

([0, 0, 4, 4], array([[2, 2],
        [8, 3]]))

Cluster the synopsis (represented as TfIdf vectors) using Kmeans. You can play with different number of centers and maximum iterations to get different results.

In [170]:
%time (labels, centers) = Kmeans(tfidf_matrix.todense(), num_clusters=3, max_iter=1000)

ValueError: operands could not be broadcast together with shapes (2,) (473,) 

Now we create a dataframe `frame` that stores the the clusters labels, names and genres for all the 100 movies.

In [133]:
import pandas as pd
films = {'title': titles, 'synopsis': synopsis, 'cluster': labels, 'genre': genres}
frame = pd.DataFrame(films, index = [labels] , columns = ['title', 'cluster', 'genre'])
frame

,title,cluster,genre
0,The Godfather,0,"[u' Crime', u' Drama']"
0,The Shawshank Redemption,0,"[u' Crime', u' Drama']"
0,Schindler's List,0,"[u' Biography', u' Drama', u' History']"
9,Raging Bull,9,"[u' Biography', u' Drama', u' Sport']"
9,Casablanca,9,"[u' Drama', u' Romance', u' War']"
9,One Flew Over the Cuckoo's Nest,9,[u' Drama']
9,Gone with the Wind,9,"[u' Drama', u' Romance', u' War']"
9,Citizen Kane,9,"[u' Drama', u' Mystery']"
9,The Wizard of Oz,9,"[u' Adventure', u' Family', u' Fantasy', u' Mu..."
9,Titanic,9,"[u' Drama', u' Romance']"


Movie counts for a particular cluster.

In [134]:
frame['cluster'].value_counts()

9    97
0     3
Name: cluster, dtype: int64

### Interpreting Results
We sort the cluster centers to get the most important terms per cluster and store it in `cluster_names`. We print them along with the movies in that cluster and look how well has KMeans worked.

In [157]:
print("Top terms per cluster:")
order_centroids = np.asarray(centers).argsort()[:, ::-1]
cluster_names = []
for i in range(order_centroids.shape[0]):
    print("Cluster %d words:" % i, end='')
    q = ""
    for ind in order_centroids[i, :6]:
        print(' %s' % terms[ind], end=',')
        q += str(terms[ind])
        q += " "
    cluster_names.append(q)
    print()
    print("Cluster %d titles:" % i, end='')
    for title in frame.loc[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print()
    print()

Top terms per cluster:
Cluster 0 words: 000, able,
Cluster 0 titles: The Godfather, The Shawshank Redemption, Schindler's List,

Cluster 1 words: 000, able,
Cluster 1 titles:

KeyError: 'the label [1] is not in the [index]'

### Plotting the Data
Now we plot the various movie clusters.
Basically we scale the multi-dimentional feature vector by applying 2 dimensional PCA. It is a technique used to visualize multi-dimensional plots in 2 dimensions. More about it [here](http://www.apnorton.com/blog/2016/12/19/Visualizing-Multidimensional-Data-in-Python/).

In [136]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as sklearnPCA

In [137]:
pca = sklearnPCA(n_components=2) #2-dimensional PCA
transformed = pd.DataFrame(pca.fit_transform(tfidf_matrix.todense()))

In [158]:
colors = ['red', 'blue', 'green', 'yellow', 'black', 'gray', 'orange', 'brown']
for i in range(len(cluster_names)):
    plt.scatter(transformed[labels == i][0], transformed[labels == i][1], label=cluster_names[i], c=colors[i])
plt.legend()
plt.show()

KeyError: False

# And you're done!